# MPQP Database Interaction

This notebook demonstrates how to manage quantum circuit execution results using the `mpqp` library. It includes setting up the database, running quantum circuits on simulators, storing results, and performing database queries.

In [1]:
from mpqp.all import *
from mpqp.execution.connection.env_manager import get_env_variable

## Database Setup

In [2]:
from mpqp.db.db_setup import setup_db

To store and retrieve circuit execution results, we need a database. The `setup_db` function initializes this database.

by default `setup_db` will be called `quantum_jobs.db`. We can change this by giving the database name as an argument
The name of the database will be stored in the global variable of mpqp.

In [3]:
setup_db()
print(get_env_variable("DATA_BASE"))

quantum_jobs.db


In [4]:
setup_db("another_database.db")
print(get_env_variable("DATA_BASE"))

another_database.db


## Insert Results into the Database

To save circuit execution results, use the `insert_result` function from `mpqp.db.db_import`.

In [5]:
from mpqp.db.db_import import insert_result

We create a simple quantum circuit using gates and run it on a simulator.
We can then store the results in the database.

By default if a job already exists the result will put the same job_id.

In [6]:
c = QCircuit([H(0), CNOT(0, 1), BasisMeasure()], label="H CX BM")

result = run(c, device=IBMDevice.AER_SIMULATOR)

id = insert_result(result)
print(id)

[7]


We can also store a `BatchResult`.

In [7]:
result2 = run(c, device=[IBMDevice.AER_SIMULATOR, GOOGLEDevice.CIRQ_LOCAL_SIMULATOR])

id2 = insert_result(result2)
print(id2)

[8, 9]


## Query the Database

The `mpqp.db.db_query` module provides functions to fetch results and jobs.

In [8]:
from mpqp.db.db_query import *

assert isinstance(result, Result)
job = result.job

Retrieve all results stored in the database.

In [9]:
all_results = fetch_all_results()
all_results

[{'id': 1,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=520, probability=0.5078125), Sample(2, index=3, count=504, probability=0.4921875)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:51:13'},
 {'id': 2,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=491, probability=0.4794921875), Sample(2, index=3, count=533, probability=0.5205078125)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:51:14'},
 {'id': 3,
  'job_id': 2,
  'data': '"[Sample(2, index=0, count=506, probability=0.494140625), Sample(2, index=3, count=518, probability=0.505859375)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:51:14'},
 {'id': 4,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=493, probability=0.4814453125), Sample(2, index=3, count=531, probability=0.5185546875)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:54:55'},
 {'id': 5,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=511, probability=0.49902

Retrieve all jobs stored in the database.


In [10]:
all_jobs = fetch_all_jobs()
all_jobs

[{'id': 1,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure()"',
  'created_at': '2024-11-27 12:51:13'},
 {'id': 2,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label=\\"H CX BM\\")"',
  'device': 'GOOGLEDevice.CIRQ_LOCAL_SIMULATOR',
  'measure': '"BasisMeasure()"',
  'created_at': '2024-11-27 12:51:14'}]

2 results have the same job so it is expected to have only 2 jobs instead of 3

Retrieve specific result using their ID.

In [11]:
db_result_id_1 = fetch_result_with_id(1)
db_result_id_1

{'id': 1,
 'job_id': 1,
 'data': '"[Sample(2, index=0, count=520, probability=0.5078125), Sample(2, index=3, count=504, probability=0.4921875)]"',
 'error': None,
 'shots': 1024,
 'created_at': '2024-11-27 12:51:13'}

Retrieve specific job using their ID.

In [12]:
assert db_result_id_1 is not None
db_job_id_1 = fetch_job_with_id(db_result_id_1['job_id'])
db_job_id_1

{'id': 1,
 'type': 'SAMPLE',
 'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label=\\"H CX BM\\")"',
 'device': 'IBMDevice.AER_SIMULATOR',
 'measure': '"BasisMeasure()"',
 'created_at': '2024-11-27 12:51:13'}

Retrieve all results using the job ID.

In [13]:
db_jobs_with_results = fetch_results_with_job_id(1)
db_jobs_with_results

[{'id': 1,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=520, probability=0.5078125), Sample(2, index=3, count=504, probability=0.4921875)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:51:13'},
 {'id': 2,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=491, probability=0.4794921875), Sample(2, index=3, count=533, probability=0.5205078125)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:51:14'},
 {'id': 4,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=493, probability=0.4814453125), Sample(2, index=3, count=531, probability=0.5185546875)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:54:55'},
 {'id': 5,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=511, probability=0.4990234375), Sample(2, index=3, count=513, probability=0.5009765625)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:54:55'},
 {'id': 7,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=486, probability=0.474

Retrieve all jobs associated with a job.

In [14]:
db_jobs_with_job = fetch_jobs_with_job(job)
db_jobs_with_job

[{'id': 1,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure()"',
  'created_at': '2024-11-27 12:51:13'}]

Retrieve all jobs associated with a result.

In [15]:
db_jobs_with_results = fetch_jobs_with_result(result)
db_jobs_with_results

[{'id': 1,
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure()"',
  'created_at': '2024-11-27 12:51:13',
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=486, probability=0.474609375), Sample(2, index=3, count=538, probability=0.525390625)]"',
  'error': None,
  'shots': 1024}]

Retrieve all results associated with a result.

In [16]:
db_results_with_results = fetch_results_with_result(result)
db_results_with_results

[{'id': 7,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=486, probability=0.474609375), Sample(2, index=3, count=538, probability=0.525390625)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:55:02'}]

Retrieve all results associated with a result and result.job.

In [17]:
db_results_with_results_and_job = fetch_results_with_result_and_job(result)
db_results_with_results_and_job

[{'id': 7,
  'job_id': 1,
  'data': '"[Sample(2, index=0, count=486, probability=0.474609375), Sample(2, index=3, count=538, probability=0.525390625)]"',
  'error': None,
  'shots': 1024,
  'created_at': '2024-11-27 12:55:02',
  'type': 'SAMPLE',
  'circuit': '"QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label=\\"H CX BM\\")"',
  'device': 'IBMDevice.AER_SIMULATOR',
  'measure': '"BasisMeasure()"'}]

## Export Data

You can also export database entries using the `mpqp.db.db_export` module.

In [18]:
from mpqp.db.db_export import *

transform a db format jobs to mpqp format.

In [19]:
jobs_dict_to_mpqp(db_job_id_1)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure())]

transform a db format results to mpqp format.

In [20]:
print(results_dict_to_mpqp(db_result_id_1)[0])

Result: H CX BM, IBMDevice, AER_SIMULATOR
 Counts: [520, 0, 0, 504]
 Probabilities: [0.50781, 0, 0, 0.49219]
 Samples:
  State: 00, Index: 0, Count: 520, Probability: 0.5078125
  State: 11, Index: 3, Count: 504, Probability: 0.4921875
 Error: None


Fetch all jobs

In [21]:
get_all_jobs()

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure()),
 Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), GOOGLEDevice.CIRQ_LOCAL_SIMULATOR, BasisMeasure())]

Fetch all results

In [22]:
get_all_results()

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure()), [Sample(2, index=0, count=520, probability=0.5078125), Sample(2, index=3, count=504, probability=0.4921875)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure()), [Sample(2, index=0, count=491, probability=0.4794921875), Sample(2, index=3, count=533, probability=0.5205078125)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), GOOGLEDevice.CIRQ_LOCAL_SIMULATOR, BasisMeasure()), [Sample(2, index=0, count=506, probability=0.494140625), Sample(2, index=3, count=518, probability=0.505859375)], None, 1024),
 Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULAT

In [23]:
get_jobs_with_job(job)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure())]

In [24]:
get_jobs_with_result(result)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure())]

In [25]:
get_results_with_result_and_job(result)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure()), [Sample(2, index=0, count=486, probability=0.474609375), Sample(2, index=3, count=538, probability=0.525390625)], None, 1024)]

In [26]:
get_results_with_result(result)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure()), [Sample(2, index=0, count=486, probability=0.474609375), Sample(2, index=3, count=538, probability=0.525390625)], None, 1024)]

In [27]:
get_result_with_id(1)

[Result(Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure()), [Sample(2, index=0, count=520, probability=0.5078125), Sample(2, index=3, count=504, probability=0.4921875)], None, 1024)]

In [28]:
get_job_with_id(1)

[Job(JobType.SAMPLE, QCircuit([H(0), CNOT(0, 1), BasisMeasure()], nb_qubits=2, nb_cbits=2, label="H CX BM"), IBMDevice.AER_SIMULATOR, BasisMeasure())]